Notebook explaining step by step how we build the `compute_likelihood_map` function.

In [1]:
import retinoto_py as fovea
args = fovea.Params()
args

Welcome on macOS-26.1-arm64-arm-64bit-Mach-O	- Timestamp (UTC)  2025-12-03_18-27-40	 user laurentperrinet	Running on MPS device (Apple Silicon/MacOS)	 - macos_version = 26.1	 with device mps, pytorch==2.9.1
Random seed 2018 has been set.


Params(image_size=224, do_mask=False, do_fovea=False, rs_min=0.0, rs_max=-5.0, padding_mode='zeros', seed=2018, batch_size=256, num_workers=0, in_memory=True, model_name='convnext_base', do_scratch=False, num_epochs=20, n_train_stop=65536, n_val_stop=16384, lr=3e-06, delta1=0.05, delta2=0.01, weight_decay=0.02, label_smoothing=0.03, shuffle=True, verbose=True)

In [2]:
import torch
from torchvision.io import read_image

alpha = .5
s_max = 1000

In [3]:
idx_to_label = fovea.get_idx_to_label(args)
label2idx = fovea.get_label_to_idx(args)
label2idx['impala']

Loading labels from local cache cached_data/imagenet_class_index.json...
Loading labels from local cache cached_data/imagenet_class_index.json...


352

In [4]:
for key in label2idx.keys(): 
    if 'owl' in key: print(key)

great_grey_owl
howler_monkey
mixing_bowl
soup_bowl


In [5]:
dataset = 'bbox'
model_filename = args.data_cache / f'33_fovea_model_name={args.model_name}_dataset={dataset}.pth'
model = fovea.load_model(args, model_filename=model_filename)
model_filename

loading .... cached_data/33_fovea_model_name=convnext_base_dataset=bbox.pth


PosixPath('cached_data/33_fovea_model_name=convnext_base_dataset=bbox.pth')

In [6]:
from torchvision.transforms.functional import InterpolationMode, resize
image_size_full = 1024
resolution = (50, 50)
size_ratio = .6

for image_url in  [
               'wolf',
               'fluffy-white-clouds',
               'fractal',
            #    'farm_animals', # to debug
               'jwst-carina-nebula',
               'L3HYK4hkM2gUTjBQbBMVkG', # mars rover
               'Nanedi_Valles_valley_system_on_Mars_ESA199848',
               'my_jackson_pollock_painting_by_amau41200-d4vjeut',
               'JamesWebbSpaceTelescope',
               'green-jungle-trees-plants-hd-1080P-wallpaper',
                                ]:
    print(f'{image_url=}')
    full_image = read_image('./images/' + image_url + '.jpg')[:3, :, :]/255
    full_image = resize(full_image, image_size_full, interpolation=InterpolationMode.BILINEAR, antialias=True)
    print(f"{type(full_image) = }, {full_image.dtype = }, {full_image.shape = }")

    args = fovea.Params(do_fovea=True)

    pos_H, pos_W, outputs = fovea.compute_likelihood_map(args, model, full_image, size_ratio=size_ratio, resolution=resolution)
    outputs = outputs.cpu()
    logits, preds = torch.max(outputs, dim=1)

    for logit, pred in zip(logits, preds):
        print(f'{logit.item():.3f} - {idx_to_label[pred]} ({pred})', end ='\t')
    print(f'>>>> {logits[torch.argmax(logits)].item():.3f} - {idx_to_label[preds[torch.argmax(logits)]]} ({preds[torch.argmax(logits)]})', end ='\t')

    print(50*'.-')




image_url='wolf'
type(full_image) = <class 'torch.Tensor'>, full_image.dtype = torch.float32, full_image.shape = torch.Size([3, 1024, 1820])


RuntimeError: MPS backend out of memory (MPS allocated: 24.55 GiB, other allocations: 7.31 GiB, max allowed: 42.43 GiB). Tried to allocate 14.61 GiB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [ ]:

for image_url, true_label in  [
               ('fluffy-white-clouds', 'parachute'),
               ('fractal', 'parachute'),
               ('jwst-carina-nebula', 'parachute'),
               ('L3HYK4hkM2gUTjBQbBMVkG', 'Arabian_camel'), 
               ('Nanedi_Valles_valley_system_on_Mars_ESA199848', 'parachute'),
               ('my_jackson_pollock_painting_by_amau41200-d4vjeut', 'parachute'),
               ('JamesWebbSpaceTelescope'), 'chocolate_sauce',
               ('green-jungle-trees-plants-hd-1080P-wallpaper', 'spider_web'),
                                ]:
    full_image = read_image('./images/' + image_url + '.jpg')[:3, :, :]/255
    full_image = resize(full_image, image_size_full, interpolation=InterpolationMode.BILINEAR, antialias=True)
    print(f"{type(full_image) = }, {full_image.dtype = }, {full_image.shape = }")

    args = fovea.Params(do_fovea=True)

    pos_H, pos_W, outputs = fovea.compute_likelihood_map(args, model, full_image, size_ratio=.3)
    outputs = outputs.cpu()
    logits, preds = torch.max(outputs, dim=1)
    for logit, pred in zip(logits, preds):
        print(f'{logit.item():.3f} - {idx_to_label[pred]}', end ='\t')


    fig, ax = fovea.plt.subplots()
    full_image_np = torch.movedim(full_image, (1, 2, 0), (0, 1, 2)).numpy()
    ax.imshow(full_image_np)
    ax.scatter(pos_W.ravel(), pos_H.ravel(), s=logit_label.abs()*s_max, c=logit_label, alpha=alpha, edgecolors='none', cmap='coolwarm', vmin=-max(abs(logit_label)), vmax=max(abs(logit_label)),)

    ax.scatter(pos_W.ravel()[idx_max], pos_H.ravel()[idx_max], s=logit_label[idx_max]*s_max, marker='*', c='white', alpha=alpha)
    fig.set_facecolor(color='white')
    fovea.plt.show()